In [4]:
import os 
import shutil
import pickle
import datetime
import numpy as np
import pandas as pd
from datetime import date, timedelta

import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Load Raw Data

### Customer Info

In [5]:
# customers
path = '../data/processed'
customers = pd.read_pickle(os.path.join(path,'customers.pkl'))
customers.head(2)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0,0.0000,0.0000,ACTIVE,NONE,49.0000,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,1,0.0000,0.0000,ACTIVE,NONE,25.0000,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...


3

In [5]:
# def chunker(seq,size):
#     return (seq[pos:pos + size] for pos in range(0,len(seq),size))
# for i in chunker(customers,1000):
#     print(i)

     customer_id     FN  Active club_member_status fashion_news_frequency  \
0              0 0.0000  0.0000             ACTIVE                   NONE   
1              1 0.0000  0.0000             ACTIVE                   NONE   
2              2 0.0000  0.0000             ACTIVE                   NONE   
3              3 0.0000  0.0000             ACTIVE                   NONE   
4              4 1.0000  1.0000             ACTIVE              Regularly   
..           ...    ...     ...                ...                    ...   
995          995 1.0000  1.0000             ACTIVE              Regularly   
996          996 0.0000  0.0000             ACTIVE                   NONE   
997          997 0.0000  0.0000             ACTIVE                   NONE   
998          998 0.0000  0.0000             ACTIVE                   NONE   
999          999 0.0000  0.0000             ACTIVE                   NONE   

        age                                        postal_code  
0   49.000

       customer_id     FN  Active club_member_status fashion_news_frequency  \
65000        65000 0.0000  0.0000             ACTIVE                   NONE   
65001        65001 0.0000  0.0000             ACTIVE                   NONE   
65002        65002 0.0000  0.0000             ACTIVE                   NONE   
65003        65003 1.0000  1.0000             ACTIVE              Regularly   
65004        65004 1.0000  1.0000             ACTIVE              Regularly   
...            ...    ...     ...                ...                    ...   
65995        65995 0.0000  0.0000             ACTIVE                   NONE   
65996        65996 1.0000  1.0000             ACTIVE              Regularly   
65997        65997 1.0000  1.0000             ACTIVE              Regularly   
65998        65998 0.0000  0.0000             ACTIVE                   NONE   
65999        65999 0.0000  0.0000             ACTIVE                   NONE   

          age                                      

        customer_id     FN  Active club_member_status fashion_news_frequency  \
131000       131000 0.0000  0.0000             ACTIVE                   NONE   
131001       131001 1.0000  1.0000             ACTIVE              Regularly   
131002       131002 0.0000  0.0000             ACTIVE                   NONE   
131003       131003 1.0000  1.0000             ACTIVE              Regularly   
131004       131004 0.0000  0.0000             ACTIVE                   NONE   
...             ...    ...     ...                ...                    ...   
131995       131995 1.0000  1.0000             ACTIVE              Regularly   
131996       131996 0.0000  0.0000             ACTIVE                   NONE   
131997       131997 0.0000  0.0000             ACTIVE                   NONE   
131998       131998 0.0000  0.0000             ACTIVE                   NONE   
131999       131999 0.0000  0.0000             ACTIVE                   NONE   

           age                         

[1000 rows x 7 columns]
        customer_id     FN  Active club_member_status fashion_news_frequency  \
204000       204000 1.0000  1.0000             ACTIVE              Regularly   
204001       204001 0.0000  0.0000             ACTIVE                   NONE   
204002       204002 1.0000  1.0000             ACTIVE              Regularly   
204003       204003 0.0000  0.0000             ACTIVE                   NONE   
204004       204004 0.0000  0.0000             ACTIVE                   NONE   
...             ...    ...     ...                ...                    ...   
204995       204995 0.0000  0.0000             ACTIVE                   NONE   
204996       204996 1.0000  1.0000             ACTIVE              Regularly   
204997       204997 1.0000  1.0000             ACTIVE              Regularly   
204998       204998 1.0000  1.0000             ACTIVE              Regularly   
204999       204999 0.0000  0.0000             ACTIVE                   NONE   

           age 

[1000 rows x 7 columns]
        customer_id     FN  Active club_member_status fashion_news_frequency  \
270000       270000 0.0000  0.0000             ACTIVE                   NONE   
270001       270001 1.0000  1.0000             ACTIVE              Regularly   
270002       270002 0.0000  0.0000             ACTIVE                   NONE   
270003       270003 0.0000  0.0000             ACTIVE                   NONE   
270004       270004 1.0000  1.0000             ACTIVE              Regularly   
...             ...    ...     ...                ...                    ...   
270995       270995 0.0000  0.0000         PRE-CREATE                   NONE   
270996       270996 0.0000  0.0000             ACTIVE                   NONE   
270997       270997 0.0000  0.0000             ACTIVE                   NONE   
270998       270998 1.0000  1.0000             ACTIVE              Regularly   
270999       270999 0.0000  0.0000             ACTIVE                   NONE   

           age 

...

         customer_id     FN  Active club_member_status fashion_news_frequency  \
1056000      1056000 0.0000  0.0000             ACTIVE                   NONE   
1056001      1056001 0.0000  0.0000             ACTIVE                   NONE   
1056002      1056002 0.0000  0.0000             ACTIVE                   NONE   
1056003      1056003 0.0000  0.0000         PRE-CREATE                   NONE   
1056004      1056004 0.0000  0.0000             ACTIVE                   NONE   
...              ...    ...     ...                ...                    ...   
1056995      1056995 0.0000  0.0000             ACTIVE                   NONE   
1056996      1056996 0.0000  0.0000             ACTIVE                   NONE   
1056997      1056997 0.0000  0.0000             ACTIVE                   NONE   
1056998      1056998 0.0000  0.0000             ACTIVE                   NONE   
1056999      1056999 1.0000  1.0000             ACTIVE              Regularly   

            age            

[1000 rows x 7 columns]
         customer_id     FN  Active club_member_status fashion_news_frequency  \
1122000      1122000 1.0000  1.0000             ACTIVE              Regularly   
1122001      1122001 0.0000  0.0000             ACTIVE                   NONE   
1122002      1122002 0.0000  0.0000             ACTIVE                   NONE   
1122003      1122003 0.0000  0.0000             ACTIVE                   NONE   
1122004      1122004 0.0000  0.0000             ACTIVE                   NONE   
...              ...    ...     ...                ...                    ...   
1122995      1122995 0.0000  0.0000             ACTIVE                   NONE   
1122996      1122996 1.0000  1.0000             ACTIVE              Regularly   
1122997      1122997 1.0000  1.0000             ACTIVE              Regularly   
1122998      1122998 0.0000  0.0000             ACTIVE                   NONE   
1122999      1122999 0.0000  0.0000             ACTIVE                   NONE   

   

[1000 rows x 7 columns]
         customer_id     FN  Active club_member_status fashion_news_frequency  \
1190000      1190000 0.0000  0.0000             ACTIVE                   NONE   
1190001      1190001 0.0000  0.0000             ACTIVE                   NONE   
1190002      1190002 1.0000  1.0000             ACTIVE              Regularly   
1190003      1190003 1.0000  1.0000             ACTIVE              Regularly   
1190004      1190004 0.0000  0.0000             ACTIVE                   NONE   
...              ...    ...     ...                ...                    ...   
1190995      1190995 1.0000  1.0000             ACTIVE              Regularly   
1190996      1190996 0.0000  0.0000             ACTIVE                   NONE   
1190997      1190997 0.0000  0.0000             ACTIVE                   NONE   
1190998      1190998 0.0000  0.0000             ACTIVE                   NONE   
1190999      1190999 0.0000  0.0000         PRE-CREATE                   NONE   

   

[1000 rows x 7 columns]
         customer_id     FN  Active club_member_status fashion_news_frequency  \
1258000      1258000 0.0000  0.0000             ACTIVE                   NONE   
1258001      1258001 0.0000  0.0000             ACTIVE                   NONE   
1258002      1258002 0.0000  0.0000             ACTIVE                   NONE   
1258003      1258003 0.0000  0.0000             ACTIVE                   NONE   
1258004      1258004 1.0000  1.0000             ACTIVE              Regularly   
...              ...    ...     ...                ...                    ...   
1258995      1258995 0.0000  0.0000             ACTIVE                   NONE   
1258996      1258996 0.0000  0.0000             ACTIVE                   NONE   
1258997      1258997 0.0000  0.0000             ACTIVE                   NONE   
1258998      1258998 0.0000  0.0000             ACTIVE                   NONE   
1258999      1258999 0.0000  0.0000             ACTIVE                   NONE   

   

[1000 rows x 7 columns]
         customer_id     FN  Active club_member_status fashion_news_frequency  \
1328000      1328000 1.0000  1.0000             ACTIVE              Regularly   
1328001      1328001 0.0000  0.0000             ACTIVE                   NONE   
1328002      1328002 1.0000  1.0000             ACTIVE              Regularly   
1328003      1328003 0.0000  0.0000             ACTIVE                   NONE   
1328004      1328004 1.0000  1.0000             ACTIVE              Regularly   
...              ...    ...     ...                ...                    ...   
1328995      1328995 0.0000  0.0000             ACTIVE                   NONE   
1328996      1328996 0.0000  0.0000         PRE-CREATE                   NONE   
1328997      1328997 0.0000  0.0000             ACTIVE                   NONE   
1328998      1328998 0.0000  0.0000             ACTIVE                   NONE   
1328999      1328999 0.0000  0.0000             ACTIVE                   NONE   

   

### Index to Customer_ID

In [14]:
# mapping index
path = '../data/processed'
infile = open(os.path.join(path,'index_to_cusId.pkl'),'rb')
index_to_id_dict = pickle.load(infile)
infile.close()

In [15]:
customers["customer_id"] = customers["customer_id"].map(index_to_id_dict)
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0000,0.0000,ACTIVE,NONE,49.0000,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0000,0.0000,ACTIVE,NONE,25.0000,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0000,0.0000,ACTIVE,NONE,24.0000,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.0000,0.0000,ACTIVE,NONE,54.0000,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0000,1.0000,ACTIVE,Regularly,52.0000,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


### Transaction File

In [16]:
path = '../data/processed'
trans = pd.read_pickle(os.path.join(path,'transactions.pkl'))
trans["customer_id"] = trans["customer_id"].map(index_to_id_dict)
trans.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.0508,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.0305,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.0152,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.0169,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.0169,2


### Articles Description

In [17]:
# articles description
articles = pd.read_pickle(os.path.join(path,'articles.pkl'))
articles.head(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [28]:
temp=dict(zip(articles['article_id'],articles['prod_name']))
temp['0108775015']

'Strap top'

## Select n Customer

In [18]:
# Set Number of selected Customers
n = 50

In [19]:
import names
# temp = trans[trans.t_dat<'2020-09-15']\
#             .groupby(['customer_id'])\
#             .agg({'article_id':'nunique'})\
#             .rename(columns={'article_id':'quantity'})\
#             .reset_index()

# # temp['quantity'] = temp['quantity'].clip(0,5)
# temp['quantity'] = temp['quantity'].astype('int8')
# temp = temp[temp['quantity'] <100]
# temp.sort_values(by=['quantity'],ascending=False,inplace=True)

# # Select N Customers
# cus_list = temp[['customer_id']][:n].reset_index(drop=True)

# random name
cus_name = {}
for i in range(n):
    cus_name[i] = names.get_full_name()
cus_name = pd.DataFrame.from_dict(cus_name,orient='index',columns=['name'])



cus_path = '../static/model'
select_cus = pd.read_csv(os.path.join(cus_path,'submission50_non.csv'))

# # Final list
# select_cus = customers[customers.customer_id.isin(cus_list.customer_id)].reset_index(drop=True)
select_cus = select_cus.join(cus_name)



select_cus.head()

,customer_id,prediction,recent_purchase,name
0,0971164b9c437c786a46e092ae17dd1d63894cf6d0baee...,0756662008 0428291016 0824264001 0878026002 07...,428291016,Mary Osullivan
1,0a0f5e47d56428eba1f1fda252c5e35a4202e8e687aeb9...,0554598028 0616453001 0698273001 0888488002 07...,554598001,Lizzie Morales
2,0a12965b9c32c9f72d761e6b65bc1d446ea718419c5419...,0663515005 0689563004 0562252039 0867969002 03...,413707001,Randy Kubinski
3,0c5555ac69bd3af2871f203a746667483c8b3df955901e...,0930380001 0698786004 0745058001 0658907011 06...,770211001,James Finley
4,106b273fc758b3d4d2dd1213c43580a0d8fd99f5e0253e...,0693350001 0671453002 0554757003 0823793002 07...,554757003,Lavern Cashio


## Recent Purchase

In [20]:
select_trans = trans[trans.customer_id.isin(select_cus.customer_id)]
print('Min date: ', select_trans.t_dat.min())
print('Max date: ', select_trans.t_dat.max())
print(f'Total Customers: {select_trans.customer_id.nunique()}')

recent_purchase = pd.DataFrame(select_trans.groupby(['customer_id'])['article_id'].apply(lambda x: list(set(x))))\
                        .reset_index()\
                        .rename(columns={'article_id':'recent_purchase'})
recent_purchase['recent_purchase'] = recent_purchase['recent_purchase'].apply(lambda x: x[:12])
recent_purchase.head()

Min date:  2018-09-20 00:00:00
Max date:  2020-09-22 00:00:00
Total Customers: 50


,customer_id,recent_purchase
0,0971164b9c437c786a46e092ae17dd1d63894cf6d0baee...,"[0680790003, 0653145007, 0798966003, 079179500..."
1,0a0f5e47d56428eba1f1fda252c5e35a4202e8e687aeb9...,"[0555326010, 0698317007, 0570781006, 039922303..."
2,0a12965b9c32c9f72d761e6b65bc1d446ea718419c5419...,"[0711032001, 0806269003, 0572957005, 079632200..."
3,0c5555ac69bd3af2871f203a746667483c8b3df955901e...,"[0751471023, 0862340001, 0628930004, 068887300..."
4,106b273fc758b3d4d2dd1213c43580a0d8fd99f5e0253e...,"[0573085005, 0842921001, 0852773002, 067121701..."


## Prediction

In [21]:
submit = pd.read_csv('submissions.csv',dtype=str)
submit = submit[submit.customer_id.isin(select_cus.customer_id)].reset_index(drop=True)
print('total customers: ' ,len(submit))
submit.head()

total customers:  50


,customer_id,prediction
0,0971164b9c437c786a46e092ae17dd1d63894cf6d0baee...,0906169002 0925246001 0863595005 0929275001 07...
1,0a0f5e47d56428eba1f1fda252c5e35a4202e8e687aeb9...,0112679048 0111609001 0111593001 0111586001 01...
2,0a12965b9c32c9f72d761e6b65bc1d446ea718419c5419...,0610776002 0877666001 0610776105 0610776001 09...
3,0c5555ac69bd3af2871f203a746667483c8b3df955901e...,0112679048 0111609001 0111593001 0111586001 01...
4,106b273fc758b3d4d2dd1213c43580a0d8fd99f5e0253e...,0112679048 0111609001 0111593001 0111586001 01...


In [23]:
submit['customer_id'][2]

'0a12965b9c32c9f72d761e6b65bc1d446ea718419c5419d548bf15855e318a62'

In [26]:
submit['prediction'][submit.customer_id=='0a12965b9c32c9f72d761e6b65bc1d446ea718419c5419d548bf15855e318a62'].values[0]

'0610776002 0877666001 0610776105 0610776001 0931769003 0897108001 0803757014 0881942001 0827968004 0870531001 0685813001 0904584008'

In [11]:
submit.groupby(['prediction']).agg({'customer_id':'count'}).sort_values(by='customer_id',ascending=False)

,customer_id
prediction,
0112679048 0111609001 0111593001 0111586001 0111565003 0111565001 0110065011 0110065002 0110065001 0108775051 0108775044 0108775015,18
0870328003 0841383002 0873274002 0706016019 0904567002 0568597006 0882899003 0783346018 0640021019 0824995001 0898918002 0717490008,1
0920610005 0920610002 0827635002 0920610001 0720504008 0677930037 0874113005 0873771002 0871997002 0904545002 0805000007 0685814003,1
0918292001 0919365008 0748355003 0868823008 0856270002 0868823007 0868823011 0919365003 0914537001 0898439001 0898692006 0572998013,1
0917293003 0917293004 0921266001 0901330002 0805000001 0685813043 0938804001 0748566027 0876009002 0677930023 0865076001 0906305001,1
0916468002 0554450036 0554450001 0765743007 0829643003 0902419001 0554450046 0868134001 0685814022 0554450043 0911214001 0673677023,1
0915529003 0865929014 0904416001 0904416002 0865929007 0574109042 0902388001 0911870002 0865929004 0610776001 0907409001 0568601043,1
0914441001 0706016019 0914441005 0865929002 0818754001 0574109011 0818754004 0762143001 0767473009 0902518005 0574109039 0640021019,1
0909014001 0871997001 0871997002 0915611004 0855239001 0916000003 0677930077 0912095007 0898692006 0903473001 0910824001 0815808001,1


## Unique Articles

In [18]:
unique_articles = []
for i in submit.prediction:
    unique_articles = unique_articles + i.split()
    
for i in recent_purchase.recent_purchase:
    unique_articles = unique_articles + i 


## Get image to new directory

In [19]:
from tqdm import tqdm
from PIL import Image

save_path = '../static/model/images'
raw_path = '../data/raw/images'

width = 1166
hieght = 1750
reduce = 0.3

# Clear all file in the directory
for f in os.listdir(save_path):
    os.remove(os.path.join(save_path, f))

# Copy using picture
for i in unique_articles:
    src = os.path.join(raw_path,i[:3],i+'.jpg')
    dst = os.path.join(save_path,i+'.jpg')
    if os.path.exists(src):
        shutil.copyfile(src, dst)
    else:
        print(f'Missing articles: {i}')

# Resize Image
multiple_images = os.listdir(save_path)

# Looping over all of the images:
for image in multiple_images:
    img = Image.open(os.path.join(save_path, image))
    img.thumbnail(size=(width*reduce,hieght*reduce))
#     print(img)
    # We would run the command below to save the images:
    img.save(os.path.join(save_path, image), optimize=True)
print('Done. Resize Image')

Missing articles: 0610776002
Missing articles: 0610776001
Missing articles: 0179208001
Missing articles: 0610776001
Missing articles: 0179208001
Missing articles: 0610776001
Missing articles: 0408875001
Missing articles: 0408875001
Missing articles: 0408875001
Missing articles: 0901588001
Missing articles: 0408875001
Done. Resize Image


## Selected Articles Info

In [20]:
select_articles = articles[articles.article_id.isin(unique_articles)].reset_index(drop=True)
print('Total Selected Articles: ',len(select_articles))
select_articles.head()

Total Selected Articles:  841


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


## Save Selected Files

In [36]:
select_articles['prod_name'][select_articles.article_id=='0108775015'].values[0]

'Strap top'

In [29]:
select_articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [22]:
temp = submit.merge(recent_purchase,on='customer_id',how='left')
temp['recent_purchase'] = temp['recent_purchase'].apply(lambda x: ' '.join(x))
temp.head()

,customer_id,prediction,recent_purchase
0,0971164b9c437c786a46e092ae17dd1d63894cf6d0baee...,0906169002 0925246001 0863595005 0929275001 07...,0572998007 0506166046 0752512006 0920610002 07...
1,0a0f5e47d56428eba1f1fda252c5e35a4202e8e687aeb9...,0112679048 0111609001 0111593001 0111586001 01...,0568601006 0687365001 0555326005 0570781008 05...
2,0a12965b9c32c9f72d761e6b65bc1d446ea718419c5419...,0610776002 0877666001 0610776105 0610776001 09...,0680263013 0561814002 0770383001 0690936006 06...
3,0c5555ac69bd3af2871f203a746667483c8b3df955901e...,0112679048 0111609001 0111593001 0111586001 01...,0596517001 0591334014 0903910001 0817353003 07...
4,106b273fc758b3d4d2dd1213c43580a0d8fd99f5e0253e...,0112679048 0111609001 0111593001 0111586001 01...,0636323001 0813898002 0752512006 0877137002 08...


In [23]:
final_save_path = '../static/model'

temp.to_csv(os.path.join(final_save_path,'prediction.csv'))
select_articles.to_csv(os.path.join(final_save_path,'articles.csv'))
# recent_purchase.to_csv(os.path.join(final_save_path,'recent_purchase.csv'))
select_cus.to_csv(os.path.join(final_save_path,'customers.csv'))

In [ ]:
{key:"{:,}".format(overall_data[overall_data.channel == key]['annual_accounts'].values[0]) for key in overall_data.channel}